# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 200)

import sys
import os
import re
import nltk
from sqlalchemy import create_engine
import pickle
import warnings
warnings.filterwarnings('ignore')

from scipy.stats import gmean

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin

In [2]:
# load data from database
database_filepath = "disaster_response.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
df = pd.read_sql_table(table_name,engine)

### 2. Write a tokenization function to process your text data

In [3]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.0,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.0,0.063778,0.111497,0.088267,0.015449,0.023039,0.011367,0.033377,0.045545,0.131446,0.065037,0.045812,0.050847,0.020293,0.006065,0.010795,0.004577,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.0,0.244361,0.314752,0.283688,0.123331,0.150031,0.106011,0.179621,0.208500,0.337894,0.246595,0.209081,0.219689,0.141003,0.077643,0.103338,0.067502,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
#Remove child alone field because it has all zeros only
df = df.drop(['child_alone'],axis=1)

In [5]:
# check the number of 2's in the related field
df['related'].eq(2).sum()

188

In [6]:
# Replace 2 with 1 to consider it a valid response(binary).
df['related']=df['related'].map(lambda x: 1 if x == 2 else x)

In [7]:
# Extract X and y variables from the data for the modelling
X = df['message']
#select from columns with categorical values 0 or 1
y = df.iloc[:,4:]

In [8]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    """
    function to tokenize and normalize text data
    
    Arguments:
        text -> messages to be tokenized and normalized
    Output:
        normalized -> List of tokens extracted and normalized from the messages
    """
    
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    normalized = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return normalized

In [9]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    The class for implementing Verb Extractor
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline_one = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
            
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

pipeline_two = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline_one.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      5058
               request       0.80      0.54      0.64      1100
                 offer       0.09      0.03      0.05        30
           aid_related       0.77      0.59      0.67      2719
          medical_help       0.63      0.31      0.42       504
      medical_products       0.70      0.32      0.44       309
     search_and_rescue       0.60      0.17      0.26       189
              security       0.11      0.02      0.03       112
              military       0.67      0.36      0.46       233
                 water       0.73      0.64      0.69       399
                  food       0.81      0.68      0.74       712
               shelter       0.79      0.58      0.67       585
              clothing       0.68      0.40      0.50       100
                 money       0.58      0.32      0.41       153
        missing_people       0.57      

In [13]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.95      0.88     15036
               request       0.79      0.55      0.65      3374
                 offer       0.42      0.09      0.15        88
           aid_related       0.77      0.60      0.68      8141
          medical_help       0.65      0.27      0.38      1580
      medical_products       0.70      0.32      0.44      1004
     search_and_rescue       0.67      0.23      0.34       535
              security       0.42      0.08      0.13       359
              military       0.68      0.35      0.46       627
                 water       0.77      0.66      0.71      1273
                  food       0.81      0.70      0.75      2211
               shelter       0.78      0.54      0.64      1729
              clothing       0.80      0.47      0.59       305
                 money       0.59      0.31      0.40       451
        missing_people       0.64    

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
# pipeline_one.get_params().keys()
parameters_grid = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}

#parameters_grid = {'classifier__estimator__n_estimators': [10, 20, 40]}

cv = GridSearchCV(pipeline_one, param_grid=parameters_grid, scoring='f1_micro', n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x13eb9e320>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.01, 0.02,
                                                                  0.05],
                         'classifier__estima

In [15]:
# Get the prediction values from the grid search cross validator
y_prediction_test = cv.predict(X_test)
y_prediction_train = cv.predict(X_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      5058
               request       0.61      0.39      0.47      1100
                 offer       0.00      0.00      0.00        30
           aid_related       0.81      0.18      0.30      2719
          medical_help       0.68      0.13      0.21       504
      medical_products       0.76      0.13      0.22       309
     search_and_rescue       0.56      0.15      0.24       189
              security       0.00      0.00      0.00       112
              military       0.00      0.00      0.00       233
                 water       0.57      0.85      0.68       399
                  food       0.78      0.67      0.72       712
               shelter       0.86      0.31      0.45       585
              clothing       0.67      0.28      0.39       100
                 money       0.00      0.00      0.00       153
        missing_people       0.56      

In [17]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.76      1.00      0.87     15036
               request       0.61      0.39      0.47      3374
                 offer       1.00      0.02      0.04        88
           aid_related       0.80      0.19      0.31      8141
          medical_help       0.62      0.11      0.19      1580
      medical_products       0.72      0.13      0.22      1004
     search_and_rescue       0.67      0.19      0.30       535
              security       0.00      0.00      0.00       359
              military       0.00      0.00      0.00       627
                 water       0.58      0.84      0.69      1273
                  food       0.77      0.68      0.72      2211
               shelter       0.84      0.29      0.43      1729
              clothing       0.78      0.32      0.46       305
                 money       1.00      0.01      0.02       451
        missing_people       0.70    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [18]:
#Use pipeline_two which includes StartingVerbEstimator
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = pipeline_two.fit(X_train, y_train)

y_prediction_train = pipeline_fitted.predict(X_train)
y_prediction_test = pipeline_fitted.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.94      0.88      5031
               request       0.75      0.54      0.63      1106
                 offer       0.00      0.00      0.00        34
           aid_related       0.77      0.60      0.67      2760
          medical_help       0.58      0.27      0.37       500
      medical_products       0.62      0.25      0.36       322
     search_and_rescue       0.46      0.16      0.24       190
              security       0.23      0.05      0.08       124
              military       0.64      0.38      0.48       214
                 water       0.72      0.65      0.69       401
                  food       0.80      0.67      0.73       736
               shelter       0.79      0.56      0.65       589
              clothing       0.71      0.34      0.46       119
                 money       0.62      0.35      0.45       165
        missing_people       0.57      

In [19]:
# Print classification report on training data
print('\n',classification_report(y_train.values, y_prediction_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.94      0.88     15063
               request       0.80      0.55      0.65      3368
                 offer       0.18      0.05      0.08        84
           aid_related       0.77      0.59      0.67      8100
          medical_help       0.65      0.28      0.39      1584
      medical_products       0.70      0.34      0.46       991
     search_and_rescue       0.73      0.26      0.38       534
              security       0.54      0.09      0.15       347
              military       0.70      0.37      0.48       646
                 water       0.78      0.68      0.73      1271
                  food       0.81      0.69      0.75      2187
               shelter       0.79      0.54      0.64      1725
              clothing       0.78      0.43      0.55       286
                 money       0.58      0.31      0.41       439
        missing_people       0.71    

### 9. Export your model as a pickle file

In [20]:
pickled_file = pickle.dumps('classifier.pkl')